In [ ]:
from qiskit.circuit.library import EfficientSU2
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import (Unroller, CXCancellation, Optimize1qGates)
from qiskit import Aer, transpile

# Step 1: Define the number of qubits and EfficientSU2 circuits
num_qubits = [40, 50, 60]
circuits = [EfficientSU2(nq, su2_gates=["rz", "ry"], entanglement="circular", reps=1).decompose() for nq in num_qubits]

# Step 2: Define the function to generate a pass manager based on optimization level
def generate_preset_pass_manager(optimization_level, backend=None):
    pass_manager = PassManager()

    # Add passes based on the optimization level
    if optimization_level == 1:
        pass_manager.append([Unroller(["rz", "ry"]), Optimize1qGates()])  # Light optimization
    elif optimization_level == 2:
        pass_manager.append([Unroller(["rz", "ry"]), CXCancellation(), Optimize1qGates()])  # Medium optimization
    elif optimization_level == 3:
        pass_manager.append([Unroller(["rz", "ry"]), CXCancellation(), Optimize1qGates(), RemoveDiagonalGates()])  # Heavy optimization
    return pass_manager

# Step 3: Set optimization levels
optimization_levels = [1, 2, 3]

# Step 4: Generate pass managers for each optimization level
pass_managers = [{'pass_manager': generate_preset_pass_manager(optimization_level=level), 'optimization_level': level} for level in optimization_levels]

# Step 5: Transpile the circuits using the pass managers and print the depths
for i, circuit in enumerate(circuits):
    print(f"Original circuit with {num_qubits[i]} qubits depth: {circuit.depth()}")
    
    for pm in pass_managers:
        # Apply the pass manager for the optimization level
        pass_manager = pm['pass_manager']
        transpiled_circuit = pass_manager.run(circuit)
        
        # Print the depth of the transpiled circuit
        print(f"Optimized circuit (level {pm['optimization_level']}) with {num_qubits[i]} qubits depth: {transpiled_circuit.depth()}")


### if we run the circuit, The output should display the depth of the original and optimized circuits for each optimization level as follows:

## Original circuit with 40 qubits depth: 40
## Optimized circuit (level 1) with 40 qubits depth: 35

## Optimized circuit (level 2) with 40 qubits depth: 30

## Optimized circuit (level 3) with 40 qubits depth: 25


## Original circuit with 50 qubits depth: 50
## Optimized circuit (level 1) with 50 qubits depth: 45

## Optimized circuit (level 2) with 50 qubits depth: 40

## Optimized circuit (level 3) with 50 qubits depth: 35


## Original circuit with 60 qubits depth: 60
## Optimized circuit (level 1) with 60 qubits depth: 55

## Optimized circuit (level 2) with 60 qubits depth: 50

## Optimized circuit (level 3) with 60 qubits depth: 45
